In [54]:
import pandas as pd

In [55]:
df = pd.read_csv("20171020_183001__full.log", sep='~', header=None, squeeze=True)

In [56]:
logs = df.str.extract('(\d.+)\s-\s+([^\s]+)\s-\s+([^\s]+)\s-\s+(.+)')

In [57]:
logs.head()

,0,1,2,3
0,"2017-10-20 18:30:01,459",INFO,Microgate.instruments.climateChamber,"Thermal test started, saving data to 20171020_..."
1,"2017-10-20 18:30:01,865",INFO,Microgate.instruments.climateChamber,Time Fri Oct 20 18:30:01 2017 tSet=21.20 tRead...
2,"2017-10-20 18:30:01,865",INFO,__main__,State = Off
3,"2017-10-20 18:30:01,865",INFO,__main__,State changed
4,"2017-10-20 18:30:01,866",INFO,__main__,Powering off system


In [58]:
logs.rename(columns={0:'timestamp',1:'type',2:'module',3:'message'}, inplace=True)

In [59]:
logs.head()

,timestamp,type,module,message
0,"2017-10-20 18:30:01,459",INFO,Microgate.instruments.climateChamber,"Thermal test started, saving data to 20171020_..."
1,"2017-10-20 18:30:01,865",INFO,Microgate.instruments.climateChamber,Time Fri Oct 20 18:30:01 2017 tSet=21.20 tRead...
2,"2017-10-20 18:30:01,865",INFO,__main__,State = Off
3,"2017-10-20 18:30:01,865",INFO,__main__,State changed
4,"2017-10-20 18:30:01,866",INFO,__main__,Powering off system


In [60]:
logs.type.unique()

array(['INFO', 'ERROR', 'WARNING', nan], dtype=object)

In [61]:
logs = logs.loc[~logs['type'].isna()]

In [62]:
logs.type.unique()

array(['INFO', 'ERROR', 'WARNING'], dtype=object)

In [63]:
logs.module.unique()

array(['Microgate.instruments.climateChamber', '__main__',
       'Microgate.utils.testing', 'Microgate.adopt.MGP',
       'Microgate.adopt.AOSupport', 'Microgate.adopt.AODiagBuffers'],
      dtype=object)

In [64]:
logs.loc[logs.type == 'WARNING'].groupby('message').count()

,timestamp,type,module
message,,,
ADC jump of 100 detected on brick #0,5,5,5
ADC jump of 100 detected on brick #1,12,12,12
ADC jump of 100 detected on brick #2,3,3,3
ADC jump of 100 detected on brick #3,15,15,15
ADC jump of 100 detected on brick #5,2,2,2
...,...,...,...
sleepTime=-22 is negative!!!,5,5,5
sleepTime=-43 is negative!!!,1,1,1
sleepTime=-45 is negative!!!,1,1,1


In [65]:
#change the warning message into template
#template mapping
def replace_template_warning(logs):
    mapping = [('ADC jump of \d+ detected on brick #\d+', 'ADC jump of {} detected on brick #{}'),
    ('sleepTime=-\d+ is negative!!!', 'sleepTime={} is negative')]
    for pat, replace in mapping:
        logs.loc[(logs.type == 'WARNING') & (logs.message.str.match(pat)), 'message'] = replace
    #logs.loc[logs.type == 'WARNING'].groupby('message').count()
    return logs

In [66]:
logs = replace_template_warning(logs)
logs.loc[logs.type == 'WARNING'].groupby('message').count()

,timestamp,type,module
message,,,
ADC jump of {} detected on brick #{},15119,15119,15119
Error not fatal,114,114,114
sleepTime={} is negative,14,14,14


In [67]:
def read_log(filename):
    df = pd.read_csv(filename, sep='~', header=None, squeeze=True)
    logs = df.str.extract('(\d.+)\s-\s+([^\s]+)\s-\s+([^\s]+)\s-\s+(.+)')
    logs.rename(columns={0:'timestamp',1:'type',2:'module',3:'message'}, inplace=True)
    logs = logs.loc[~logs['type'].isna()]
    return logs

In [68]:
log_2 = read_log('20171026_175308__full.log')

In [69]:
log_2.head()

,timestamp,type,module,message
0,"2017-10-26 17:53:08,105",INFO,Microgate.instruments.climateChamber,"Thermal test started, saving data to 20171026_..."
1,"2017-10-26 17:53:08,490",INFO,Microgate.instruments.climateChamber,Time Thu Oct 26 17:53:08 2017 tSet=21.80 tRead...
2,"2017-10-26 17:53:08,491",INFO,__main__,State = Off
3,"2017-10-26 17:53:08,491",INFO,__main__,State changed
4,"2017-10-26 17:53:08,491",INFO,__main__,Powering off system


In [70]:
log_2 = replace_template_warning(log_2)
log_2.loc[log_2.type == 'WARNING'].groupby('message').count()

,timestamp,type,module
message,,,
ADC jump of {} detected on brick #{},9198,9198,9198
Error not fatal,271,271,271
sleepTime={} is negative,41,41,41


In [71]:
log_3 = read_log('20191016_184359__full.log')
log_3 = replace_template_warning(log_3)
log_3.loc[log_3.type == 'WARNING'].groupby('message').count()

,timestamp,type,module
message,,,
ADC jump of {} detected on brick #{},6770,6770,6770
"FullBrickTest36 is not a valid userConf TN, fall back to sysConf TN",6,6,6
"Some variable initialization are hard-coded, should be moved to the configuration DB",126,126,126
The system works internally in modes only,126,126,126
We use default eye matrix instead of loading from hdf5 to be clean up with userConf,6,6,6
adjust to use aoSupp._verf..,126,126,126
just for developers: why the following remapping line is needed?,126,126,126
sleepTime=0 is negative!!!,52,52,52
sleepTime={} is negative,231,231,231


In [72]:
log_3.loc[log_3.type == 'ERROR'].groupby('message').count()

,timestamp,type,module
message,,,
ADC 0 on Brick 3 error 10.76% -- value 18596[bit],1,1,1
ADC 0 on Brick 3 error 11.35% -- value 22248[bit],1,1,1
ADC 0 on Brick 3 error 11.51% -- value 22206[bit],1,1,1
ADC 0 on Brick 3 error 11.71% -- value 22157[bit],1,1,1
ADC 0 on Brick 3 error 17.62% -- value 17165[bit],1,1,1
...,...,...,...
can1RErrCnt: 7,8,8,8
can1RErrCnt: 8,20,20,20
can1RErrCnt: 9,10,10,10


In [73]:
log_2.loc[log_2.type == 'ERROR'].groupby('message').count()

,timestamp,type,module
message,,,
ADC 0 error 17.55% -- value 15578[bit],1,1,1
ADC 0 error 17.83% -- value 15524[bit],1,1,1
ADC 0 error 18.85% -- value 15330[bit],1,1,1
ADC 0 error 18.93% -- value 15316[bit],1,1,1
ADC 0 error 19.35% -- value 15237[bit],1,1,1
...,...,...,...
wrong brick serial number= 111 != 127 detected on device 0,45,45,45
wrong brick serial number= 122 != 127 detected on device 0,45,45,45
wrong brick serial number= 124 != 127 detected on device 0,45,45,45


In [74]:
logs.loc[logs.type == 'ERROR'].groupby('message').count()

,timestamp,type,module
message,,,
ADC 1 error 4.00% -- value 23186[bit],2,2,2
ADC 1 error 4.01% -- value 23184[bit],1,1,1
ADC 1 error 4.01% -- value 23185[bit],2,2,2
ADC 1 error 4.02% -- value 23181[bit],1,1,1
ADC 1 error 4.04% -- value 23177[bit],2,2,2
...,...,...,...
wrong brick serial number= 118 != 127 detected on device 0,19,19,19
wrong brick serial number= 119 != 127 detected on device 0,19,19,19
wrong brick serial number= 125 != 127 detected on device 0,19,19,19


In [75]:
log_2.module.unique()

array(['Microgate.instruments.climateChamber', '__main__',
       'Microgate.utils.testing', 'Microgate.adopt.MGP',
       'Microgate.adopt.AOSupport', 'Microgate.adopt.AODiagBuffers'],
      dtype=object)

In [76]:
log_3.module.unique()

array(['__main__', 'Microgate.utils.testing', 'Microgate.adopt.UserConf',
       'Microgate.adopt.AO', 'Microgate.adopt.mirrorCommand',
       'Microgate.zpb.zpbDefs', 'Microgate.instruments.climateChamber',
       'Microgate.zpb.mgp2zpb', 'Microgate.adopt.AOSupport',
       'Microgate.adopt.AODiagBuffers'], dtype=object)

In [77]:
#change the error message into template
#template error
def replace_template_err(logs):
    mapping = [('ADC reading error = .+%', 'ADC reading error = {}%'),
    ('ADC \d+ on Brick \d+ error .+% -- value \d+\[bit\]', 'ADC {} on Brick {} error {}% -- value {}[bit]'),
    ('can1RErrCnt:\s+\d+', 'can1RErrCnt: {}'), 
    ('wrong brick serial number= \d+ != \d+ detected on device \d+', 'wrong brick serial number= {} != {} detected on device {}'),
    ('ADC \d+ error .+% -- value \d+\[bit\]', 'ADC {} error {}% -- value {}[bit]')]
    for pat, replace in mapping:
        logs.loc[(logs.type == 'ERROR') & (logs.message.str.match(pat)), 'message'] = replace
    #logs.loc[logs.type == 'WARNING'].groupby('message').count()
    return logs

In [78]:
logs = replace_template_err(logs)
logs.loc[logs.type == 'ERROR'].groupby('message').count()

,timestamp,type,module
message,,,
ADC reading error = {}%,21066,21066,21066
ADC {} error {}% -- value {}[bit],87024,87024,87024
Digital test failed,1,1,1
High speed error command detected,9,9,9
High speed test failed,9,9,9
Initialization after power on failed,2,2,2
No can messages?...,179,179,179
PIC not enabled force to IDLE,11262,11262,11262
Unexcpected error power cycling...,8,8,8


In [79]:
log_2 = replace_template_err(log_2)
log_2.loc[log_2.type == 'ERROR'].groupby('message').count()

,timestamp,type,module
message,,,
ADC reading error = {}%,21730,21730,21730
ADC {} error {}% -- value {}[bit],47675,47675,47675
Digital test failed,1,1,1
High speed error command detected,15,15,15
High speed test failed,14,14,14
Initialization after power on failed,3,3,3
No can messages?...,1255,1255,1255
PIC not enabled force to IDLE,4478,4478,4478
Unexcpected error power cycling...,35,35,35


In [80]:
log_3 = replace_template_err(log_3)
log_3.loc[log_3.type == 'ERROR'].groupby('message').count()

,timestamp,type,module
message,,,
ADC reading error = {}%,8272,8272,8272
ADC {} on Brick {} error {}% -- value {}[bit],12787,12787,12787
CAN error:,2194,2194,2194
Error in dumpStatus,1,1,1
File FullBrickTest36__UserConf.hdf5 does not exist in /home/elt/E_ELT_M4/SysConf/userConf folder,6,6,6
High speed error command detected,11,11,11
High speed test failed,11,11,11
No CAN messages?...,4,4,4
PIC not enabled force to IDLE,2839,2839,2839
